In [1]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library(patchwork)
library(forcats)
library(Polychrome)
library(grid)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [72]:
xenium.obj = qs::qread('_targets/objects/obj_merged')
merged_meta = qs::qread('_targets/objects/obj_rctd_merged_01') %>% `[[`
selected_cells = merged_meta %>% rownames
xenium.obj = xenium.obj %>% subset(cells = selected_cells) %>% AddMetaData(merged_meta)
xenium.obj


Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Seurat objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Seurat objects”
Warning message:
“Not validating FOV objects”
Warning message:
“Not validating Centroids objects”
Warning message:
“Not validating

An object of class Seurat 
400 features across 28082 samples within 4 assays 
Active assay: Xenium (100 features, 0 variable features)
 1 layer present: counts
 3 other assays present: BlankCodeword, ControlCodeword, ControlProbe
 8 spatial fields of view present: fov fov.2 fov.3 fov.4 fov.5 fov.6 fov.7 fov.8

In [73]:
create_grouping_col <- function(seurat_obj, prefix) {
  new_metadata <- case_when(str_detect(seurat_obj@meta.data$labels, fixed(prefix)) ~ seurat_obj@meta.data$labels,
                            TRUE ~ NA_character_) %>% as.factor
  new_metadata = new_metadata %>% fct_relevel(sort(na.omit(levels(new_metadata))), after = Inf)
  return(new_metadata)
}



In [74]:
plot_celltype_idp = function(seurat_obj, cell_str, fov='fov'){
    new_grouping = seurat_obj %>%
        create_grouping_col(cell_str)
    seurat_obj = AddMetaData(seurat_obj, new_grouping, col.name = 'grouping_col')

    # Get the levels of the lab_Pomc column, including the NA_character_ level
    grouping_levels <- levels(seurat_obj@meta.data$grouping_col)
    # Generate colors using the polychrome palette
    how_many_colors = length(grouping_levels)
    colors <- Polychrome::sky.colors(how_many_colors) %>% as.character %>% `[`(1:how_many_colors) # Excluding the NA_character_ level
    # Add 'gray10' for the NA_character_ level
    colors <- c(colors, "gray10")
    # Create a named vector of colors, with levels as names
    named_colors <- setNames(colors, grouping_levels)
    # Map the lab_Pomc column values to the corresponding colors
    # color_vector <- mapvalues(xenium.obj@meta.data$lab_Pomc, from = lab_Pomc_levels, to = named_colors)
    
    ggp = ImageDimPlot(seurat_obj,
                       group.by='grouping_col',
                       boundaries = 'segmentation',
                       fov = fov,
                       border.size =0.1, 
                       border.color =NA,
                       na.value='gray20', 
                       cols=named_colors, 
                       axes = FALSE,
                       size=0)
    ggp
}

In [84]:
make_idp_pdf = function(seurat_obj, cell_type, pdf_folder=""){
    if (!file.exists(pdf_folder)) {
      dir.create(pdf_folder, showWarnings = F)
    }
    a1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov')  + theme(legend.position = "none")
    a2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.2') + theme(legend.position = "none")
    b1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.3') + theme(legend.position = "none")
    b2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.4') + theme(legend.position = "none")
    c1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.5') + theme(legend.position = "none")
    c2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.6') + theme(legend.position = "none")
    d1 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.7') + theme(legend.position = "none")
    d2 = plot_celltype_idp(seurat_obj, cell_type, fov='fov.8') + theme(legend.position = "none")
    title_theme <- theme(plot.title = element_text(color = "white", size = 8, hjust = 0.5))
    # Set black background theme for each plot
    legend_theme <- theme(legend.position = "bottom",
                          legend.direction = "horizontal",
                          legend.box = "horizontal",
                          legend.text = element_text(size = 4),
                          legend.key.size = unit(2, "mm"))
    # black_background_theme <- theme(plot.background = element_rect(fill = "black"),
    #                                 panel.background = element_rect(fill = "black"))
    black_background_theme <- theme(plot.background = element_rect(fill = "black", color = 'black'),
                          panel.background = element_rect(fill = "black", color = 'black'),
                          plot.margin = margin(0, 0, 0, 0),
                          panel.grid.major = element_blank(),
                          panel.grid.minor = element_blank(),
                          panel.border = element_blank())

    a1_with_title <- a1 + theme(legend.position = "right") + ggtitle("a1_Veh_PF") + title_theme + legend_theme + black_background_theme
    a2_no_legend_with_title <- a2 + guides(scale="none") + ggtitle("a2_Veh_PF") + title_theme + black_background_theme
    b1_no_legend_with_title <- b1 + guides(scale="none") + ggtitle("b1_Veh_PF") + title_theme + black_background_theme
    b2_no_legend_with_title <- b2 + guides(scale="none") + ggtitle("b2_Veh_PF") + title_theme + black_background_theme
    c1_no_legend_with_title <- c1 + guides(scale="none") + ggtitle("c1_FGF1") + title_theme + black_background_theme
    c2_no_legend_with_title <- c2 + guides(scale="none") + ggtitle("c2_FGF1") + title_theme + black_background_theme
    d1_no_legend_with_title <- d1 + guides(scale="none") + ggtitle("d1_FGF1") + title_theme + black_background_theme
    d2_no_legend_with_title <- d2 + guides(scale="none") + ggtitle("d2_FGF1") + title_theme + black_background_theme

    # Combine the plots using patchwork with a black background
    combined_plots <- a1_with_title + c1_no_legend_with_title +
                      a2_no_legend_with_title + c2_no_legend_with_title +
                      b1_no_legend_with_title + d1_no_legend_with_title +
                      b2_no_legend_with_title + d2_no_legend_with_title +
                      plot_layout(guides = "collect", ncol = 2, heights = c(1, 1, 1, 1), widths = c(1, 1))

    # combined_plots = combined_plots & black_background_theme & legend_theme 
    combined_plots = combined_plots + plot_annotation(title=cell_type, theme=black_background_theme) + plot_annotation(theme=legend_theme) + plot_annotation(theme=title_theme)
    # Save the combined_plots to a PDF with a black background
    pdf_name = paste0(pdf_folder, cell_type, '_ct_mapping.pdf')
    pdf(pdf_name, width = 8.27, height = 11.69, bg = "black")
    # grid.newpage()
    grid.draw(combined_plots)
    dev.off()
}

In [85]:
cell_types = xenium.obj %>% `[[` %>% group_by(labels) %>% summarise(n = n()) %>% arrange(desc(n)) %>% pull(labels) %>% unique
cell_types

[1] "Astrocytes"          "Met__Prlh"           "Oligodendrocytes"   
 [4] "OPC"                 "Lepr15"              "Agrp"               
 [7] "Esr1"                "b2_Tanycytes"        "Lepr89"             
[10] "a1_Tanycytes"        "Fez1"                "Dlk1"               
[13] "Nfib_1"              "Pou6f2__Prkcq__Whrn" "Sst__Pthlh"         
[16] "Endothelial_cells"   "Gpr50__Tac2"         "Pomc__Anxa2"        
[19] "Ppp1r17__Ltbp1"      "Pomc__Ttr"           "b1_Tanycytes"       
[22] "Lepr67"              "Trh__Cxcl12"         "Sst__Unc13c"        
[25] "Ghrh"                "Th__Slc6a3"          "Htr3b"              
[28] "Microglia"           "Ppp1r17__Tafa4"      "Nfib_3"             
[31] "Kiss1__Tac2"         "a2_Tanycytes"        "Tmem215__Greb1"     
[34] "Sst__Nts"            "Lpar1_oligo"         "Nfix__Nfib"         
[37] "Reln__Lef1"          "Ependymal_cells"     "Npsr1__Epha3"       
[40] "Th__Nfib"            "Met__Pappa2"         "Arx__Nr5a2"         
[43] "Tbx19__Pirt"         "Ppp1r17"             "Nfib_2"             
[46] "Pomc__Glipr1"        "Hdc"                 "Nfix__Nfib__Hgf"    
[49] "Nfix__Nfib__Lhx6"    "Slc17a6__Trhr"       "Trh__Lef1"          
[52] "Sim1"                "Irx5__Lmx1a"         "Tmem215__Dach2"

In [ ]:
for (cell_type in cell_types){
    print(cell_type)
    make_idp_pdf(xenium.obj, cell_type, pdf_folder="ct_mappings_rctd/")
}

[1] "Astrocytes"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 23897 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 23897 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 23897 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 23897 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 23897 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Met__Prlh"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 25134 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Oligodendrocytes"


...

Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27982 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27982 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27982 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27982 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27982 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Tbx19__Pirt"


Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27983 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27983 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27983 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27983 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”
Warning message:
“Removing 27983 cells missing data for vars requested”
Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requeste

[1] "Ppp1r17"


Warning message:
“Removing 27469 cells missing data for vars requested”


In [80]:
seurat_obj = xenium.obj
cell_str = 'Astrocytes'
fov='fov.1'

new_grouping = seurat_obj %>%
        create_grouping_col(cell_str)
    seurat_obj = AddMetaData(seurat_obj, new_grouping, col.name = 'grouping_col')

    # Get the levels of the lab_Pomc column, including the NA_character_ level
    grouping_levels <- levels(seurat_obj@meta.data$grouping_col)
    # Generate colors using the polychrome palette
    how_many_colors = length(grouping_levels)
    colors <- Polychrome::sky.colors(how_many_colors) %>% as.character %>% `[`(1:how_many_colors) # Excluding the NA_character_ level
    # Add 'gray10' for the NA_character_ level
    colors <- c(colors, "gray10")
    # Create a named vector of colors, with levels as names
    named_colors <- setNames(colors, grouping_levels)
    # Map the lab_Pomc column values to the corresponding colors
    # color_vector <- mapvalues(xenium.obj@meta.data$lab_Pomc, from = lab_Pomc_levels, to = named_colors)
    
    ggp = ImageDimPlot(seurat_obj,
                       group.by='grouping_col',
                       boundaries = 'segmentation',
                       fov = fov,
                       border.size =0.1, 
                       border.color =NA,
                       na.value='gray20', 
                       cols=named_colors, 
                       axes = FALSE,
                       size=0)
    ggp

Warning message in makePalette(n, sky):
“minimal value for n is 3, returning requested palette with 3 levels
”


ERROR: Error in ImageDimPlot(seurat_obj, group.by = "grouping_col", boundaries = "segmentation", : No compatible spatial coordinates present
